In [13]:
import glob
import pandas as pd
import numpy as np
from data_loader import parse_data

In [14]:
files_train = glob.glob('../data/train/*/vehicle-position-matched-online.csv')
files_test = glob.glob('../data/test/*/vehicle-position-matched-online.csv')


In [15]:
train = parse_data(files_train)
test = parse_data(files_test)
train.head()

,Time,wayId,mean
0,2018-10-03 06:00:00,103524893,13.380000
1,2018-10-03 06:00:00,103524902,10.129259
2,2018-10-03 06:00:00,108233932,13.403333
3,2018-10-03 06:00:00,108233935,2.674286
4,2018-10-03 06:00:00,108233940,0.965758


In [16]:
time = pd.DatetimeIndex(train.Time)

#calculate the historical average
df_ha=train.groupby([time.hour,time.minute,'wayId']).mean()
#Rename the index
df_ha.index=df_ha.index.rename(['hour','minute','wayId'])
#reset the index
df_ha=df_ha.reset_index()
df_ha.head()


,hour,minute,wayId,mean
0,6,0,103524893,9.406757
1,6,0,103524902,8.975976
2,6,0,108233932,11.870000
3,6,0,108233935,5.278810
4,6,0,108233940,5.045379


In [17]:
#add a hour and minute column to the test dataframe
test['hour'] = test.Time.apply(lambda x: x.hour)
test['minute'] = test.Time.apply(lambda x: x.minute)

#merge the historical mean with the test test
test_merged = pd.merge(test, df_ha, on=['hour','minute','wayId'])[['Time','wayId','mean_x','mean_y']]
#rename columns
test_merged.columns=['Time','wayid','mean_truth','mean_prediction']
test_merged.head()

,Time,wayid,mean_truth,mean_prediction
0,2018-10-03 06:00:00,103524893,13.380000,9.406757
1,2018-10-03 06:00:00,103524902,10.129259,8.975976
2,2018-10-03 06:00:00,108233932,13.403333,11.870000
3,2018-10-03 06:00:00,108233935,2.674286,5.278810
4,2018-10-03 06:00:00,108233940,0.965758,5.045379


In [18]:
MAE = abs(test_merged['mean_truth']-test_merged['mean_prediction']).mean()
MAE

0.9939977611706159

In [19]:
test_merged2 = test_merged[test_merged['mean_truth']>0.1]
MAPE = abs((test_merged2['mean_truth']-test_merged2['mean_prediction'])/(test_merged2['mean_truth'])).mean()*100
MAPE

31.744410948358475

In [20]:
RMSE = np.sqrt(((test_merged['mean_truth']-test_merged['mean_prediction'])**2).mean())
RMSE

1.5143718428825592